In [170]:
%matplotlib inline
import pandas as pd

import plotly.tools as tls
from IPython.display import display
import plotly.plotly as py # interactive graphing
from plotly.graph_objs import Bar, Scatter, Marker, Layout
from plotly import tools
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl

import plotly
plotly.tools.set_credentials_file(username='oliviac12', api_key='rtp2rltz0x')

In [225]:
#load the table that related to click
df_c = pd.read_csv('firstable.csv')
df_c.columns = map(str.lower, df_c.columns)

In [220]:
df_c.shape

(481943, 15)

In [4]:
df_c.head()

,placement_key,country,device_category,eligible_impression_requests,total_impression_requests,filled_pages,impressions,visible_impressions,clicks,total_engagements,video_plays,autoplay_views,guaranteed_brand_safe,location_type,layout_type
0,c1687c86,US,Smartphone,241,284,1,1,1,0,0,0,0,0,below the post,single
1,c1687c86,US,Tablet,9323,9410,15,15,6,0,0,0,0,0,below the post,single
2,c1687c86,US,Unknown,0,235,0,0,0,0,0,0,0,0,below the post,single
3,c16b15c0,AU,Desktop,1,1,0,0,0,0,0,0,0,1,in-feed,single
4,c16b15c0,CA,Desktop,64,64,4,4,2,0,0,0,0,1,in-feed,single


In [44]:
##For those ad slots that doesn't have impression served by Sharethrough
df_ns = df_c[df_c['impressions'] == 0]
df_ns.shape

(316430, 15)

In [45]:
df_ss = df_c[df_c['impressions'] > 0]
df_ss.shape

(165513, 15)

In [7]:
country_counts = pd.DataFrame(df_ns.country.value_counts())
country_counts = country_counts.add_suffix('_Count').reset_index()
country_counts['share'] = (country_counts['country_Count']/float(len(df_ns))*100).round(2).astype(str) + '%'
print country_counts
# device_counts = pd.DataFrame(df_ns.device_category.value_counts())
# device_counts = device_counts.add_suffix('_Count').reset_index()
# device_counts['share'] = (device_counts['device_category_Count']/float(len(df_ns))*100).round(2).astype(str) + '%'
# print device_counts

          index  country_Count   share
0   Unsupported          64769  20.47%
1            AU          36568  11.56%
2            FR          29263   9.25%
3            DE          29073   9.19%
4            IT          26470   8.37%
5            ES          26026   8.22%
6            GB          23669   7.48%
7            BE          21941   6.93%
8            CA          20294   6.41%
9            FI          19548   6.18%
10           US          18809   5.94%


In [8]:
ss_country_counts = pd.DataFrame(df_ss.country.value_counts())
ss_country_counts = ss_country_counts.add_suffix('_Count').reset_index()
ss_country_counts['share'] = (ss_country_counts['country_Count']/float(len(df_ss))*100).round(2).astype(str) + '%'
print ss_country_counts
# ss_device_counts = pd.DataFrame(df_ss.device_category.value_counts())
# ss_device_counts = ss_device_counts.add_suffix('_Count').reset_index()
# ss_device_counts['share'] = (ss_device_counts['device_category_Count']/float(len(df_ss))*100).round(2).astype(str) + '%'
# print ss_device_counts

  index  country_Count   share
0    US          61941  37.42%
1    CA          47158  28.49%
2    GB          42390  25.61%
3    AU           5108   3.09%
4    DE           3857   2.33%
5    FR           1630   0.98%
6    ES           1031   0.62%
7    IT            990    0.6%
8    BE            801   0.48%
9    FI            607   0.37%


In [9]:
##Market Share by Country

import plotly.plotly as py
import plotly.graph_objs as go

trace0 = go.Bar(
    x=country_counts['index'].values,
    y=country_counts.country_Count.values,
    text=country_counts.share.values,
    marker=dict(
        color='rgb(61, 153, 112)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    opacity=0.6,
    name = 'Non-Sharethrough'
)
trace1 = go.Bar(
    x=ss_country_counts['index'].values,
    y=ss_country_counts.country_Count.values,
    text=ss_country_counts.share.values,
    marker=dict(
        color='rgb(0, 31, 63)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    opacity=0.6,
    name = "Sharesthrough"
)
data = [trace0, trace1]
# layout = go.Layout(
#     title='2015-01-01 Market Report',
# )
layout = go.Layout(
    barmode='group'
)
fig = go.Figure(data=data, layout=layout)
plot_url = py.plot(fig, filename='Market share by Country')

In [230]:
#fill rate
df_ss['fill_rate'] = df_ss['filled_pages']/df_ss['total_impression_requests']
#CTR clicks/impressions
df_ss['ctr'] = df_ss['clicks']/df_ss['impressions']
df_ss['ctr'].describe()
#engagement/impression = engagement rate
df_ss['engagement_rate'] = df_ss['total_engagements']/df_ss['impressions']
df_ss['engagement_rate'].describe()

count    165486.000000
mean          0.003158
std           0.029942
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: engagement_rate, dtype: float64

In [47]:
df_ss = df_ss[(df_ss['ctr'] <= 1) & (df_ss['engagement_rate'] <= 1)]
df_ss.shape

(165486, 18)

In [57]:
#visible impression by location, layout and device?
vis_imp = df_ss.groupby('country')['visible_impressions'].mean().reset_index()
enga = df_ss.groupby('country')['engagement_rate'].mean().reset_index()
request = df_ss.groupby('country')['total_impression_requests'].mean().reset_index()
device = df_ss.groupby('device_category')['engagement_rate'].mean().reset_index()
# print df_ss.groupby('device_category')['fill_rate'].mean()
# print '+++++++++++++++++++++++'
# print df_ss.groupby('location_type')['engagement_rate'].mean()
location = df_ss.groupby('location_type')['fill_rate'].mean().reset_index()
print location
# print '+++++++++++++++++++++++'
layout = df_ss.groupby('layout_type')['engagement_rate'].mean().reset_index()
print layout
# print df_ss.groupby('layout_type')['fill_rate'].mean()
Enga = df_ss.groupby(['country','device_category'])['engagement_rate'].mean().reset_index()
Enga1 = df_ss.groupby(['location_type','device_category'])['engagement_rate'].mean().reset_index()
engagementR = np.array(Enga1['engagement_rate'])
engagementRate = np.array(Enga['engagement_rate'])
engagementRate = engagementRate.reshape(10, 3)
engagementR = engagementR.reshape(5,3)

    location_type  fill_rate
0  below the post   0.233388
1         gallery   0.297457
2         in-feed   0.266381
3        mid-post   0.182826
4           other   0.213835
       layout_type  engagement_rate
0         multiple         0.003589
1  multiple_manual         0.003803
2           single         0.003036


In [93]:
##Engagement Rate Head Map cross Difference Categories


fig = tools.make_subplots(rows=2, cols=1,
                          specs=[[{}],
                                 [{}]], shared_xaxes=True,
                          shared_yaxes=False, vertical_spacing=0.001)

fig.append_trace(go.Heatmap(
        z=engagementRate,
        x=['Desktop', 'Smartphone', 'Tablet'],
        y=enga.country.values,
        colorscale=[[0, '#3D9970'], [5, '#001f3f']],
        showscale = True), 1, 1)

fig.append_trace(go.Heatmap(
        z=engagementR,
        x=['Desktop', 'Smartphone', 'Tablet'],
        y=location.location_type.values,
        colorscale=[[0, '#3D9970'], [5, '#001f3f']],
        showscale = False), 2, 1)

layout = go.Layout(
#     title='Engagement Rate cross /n Difference Categories',
    autosize=False,
    width=350,
    height=480,
    margin=go.Margin(
        pad=4
    )
)
fig['layout'].update(layout)
plot_url = py.plot(fig, filename='labelled-heatmap')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]



In [94]:
df_en = pd.read_csv('Engagement.csv')
df_en.columns = map(str.lower, df_en.columns)

In [124]:
# vis_imp = df_ss.groupby('country')['visible_impressions'].mean().reset_index()
device_social = df_en.groupby('device_category')['socialmedia_shares'].sum().reset_index()
device_social['socialmedia_shares'] = (device_social['socialmedia_shares']/10000)
device_enga = df_ss.groupby('device_category')['engagement_rate'].mean().reset_index()
device_enga['engagement_rate'] = (device_enga['engagement_rate']*100).round(2)
print device_enga

  device_category  engagement_rate
0         Desktop             0.20
1      Smartphone             0.35
2          Tablet             0.37


In [215]:
##Social Media Share and Engagement Rate in Different Devices

trace1 = go.Bar(
    x = device_social.device_category.values,
    y = device_social['socialmedia_shares'].values,
#     mode = 'lines+markers',
    name = 'Aggregate Socialmedia_shares in ten thousands ',
    marker=dict(
        color='rgb(195, 151, 162)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
)
    )
trace2 = go.Bar(
    x = device_enga.device_category.values,
    y = device_enga.engagement_rate.values,
#     mode = 'lines',
    name = 'Average engagement rate',
        marker=dict(
        color='rgb(153, 195, 194)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
)
    )
layout = dict(
legend=dict(
        x=0.029,
        y=1.138,
        font=dict(
            size=10,
        )
        )
    )
data = [trace1, trace2]
fig = go.Figure(data=data, layout=layout)
# Plot and embed in ipython notebook!
plot_url = py.plot(fig, filename='scatter-mode')

In [135]:
#load the table that related to engagement activity
df_e = pd.read_csv('Engagement.csv')
df_e.columns = map(str.lower, df_e.columns)

In [216]:
# Scatter Pairs between Social Media Share, Auto Play and Ten-seconds Play

trace1 = go.Scatter(
    x = df_e.socialmedia_shares.values,
    y = df_e.autoplay_views.values,
    mode = 'markers',
    xaxis='x1', yaxis='y1',
    marker = dict( size = 3, color = 'rgb(153, 195, 194)')
)

trace4 = go.Scatter(
    x = df_e.autoplay_views.values,
    y = df_e.ten_second_views.values,    
    mode = 'markers',
    xaxis='x4', yaxis='y4',
    marker = dict( size = 3, color = 'rgb(153, 204, 255)')
)

trace2 = go.Histogram(
        x= np.log(df_e['socialmedia_shares']) - (np.log(df_e['socialmedia_shares'])).mean(),
        xaxis='x2', yaxis='y2',
        marker = dict(color = 'rgb(153, 51, 51)')
    )


trace3 = go.Scatter(
    x = df_e.socialmedia_shares.values,
    y = df_e.ten_second_views.values,
    mode = 'markers',
    xaxis='x3', yaxis='y3',
    marker = dict( size = 3, color = 'rgb(255, 204, 204)')
)

layout = go.Layout(title = 'Styled Scatter',
              yaxis = dict(showgrid = False),
              xaxis = dict(showgrid = False),
              showlegend=False)

fig = tools.make_subplots(rows=2, cols=2,
                              specs=[[{}, {}],
                                     [{}, {}]])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)

# fig['layout']['xaxis1'].update(showgrid=False)
# fig['layout']['xaxis2'].update(showgrid=False)
fig['layout']['xaxis3'].update(title='socialmedia_shares')
fig['layout']['xaxis4'].update(title='autoplay_views', showgrid = False)

fig['layout']['yaxis1'].update(title='autoplay_views')
# fig['layout']['yaxis2'].update(showgrid=False)
fig['layout']['yaxis3'].update(title='ten_second_views')


fig['layout'].update(showlegend = False, height = 400, width = 600)



# fig = go.Figure(fig, layout=layout)
plot_url = py.plot(fig, filename='scatter pairs')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]

The draw time for this plot will be slow for all clients.
